# Entity Workshop: Github Archive (gharchive.org)

In [9]:
import pandas as pd

# hi tim this is a change to a file.  Here's another change.

In [5]:
from dask.distributed import Client

# macbook pro 14, 2021, M1 Max, 10 core
client = Client(n_workers=4)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 46.98 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39375,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 46.98 GiB
Comm: tcp://127.0.0.1:44325,Total threads: 2
Dashboard: http://127.0.0.1:42407/status,Memory: 11.74 GiB
Nanny: tcp://127.0.0.1:42841,


## Prepare Notebook 

In [6]:
%matplotlib inline
%matplotlib widget

# https://medium.com/@pablosjv/importing-local-python-modules-into-jupyter-notebooks-87336b2d7746
%load_ext autoreload
%autoreload 2

import os
import sys

# for the future.
#import modin.pandas as pd
#%env MODIN_ENGINE=dask

import pandas as pd
import numpy as np

# By convention, we import the module dask.dataframe as dd, and call the corresponding DataFrame object ddf.
# Note: The term "Dask DataFrame" is slightly overloaded. Depending on the context, it can refer to the module or the DataFrame object. To avoid confusion, throughout this notebook:
# * dask.dataframe (note the all lowercase) refers to the API, and
# * DataFrame (note the CamelCase) refers to the object.
import dask
import dask.dataframe as dd


import matplotlib.pyplot as plt
from collections import defaultdict
from IPython.display import display, HTML
from sqlalchemy import Connection, create_engine

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

# allow iteration on modules without having to restart the kernel each time
module_path = os.path.abspath(os.path.join('data/gharchive')) # or the path to your source code
sys.path.insert(0, module_path)

# load helpers
%run data/gharchive/gharchive.py
config = GHARCHIVE_ENTITY_CONFIG

# load db helpers
# %run data/gharchive/pandas2db.py

# data downloaded from gharchive.
SEED_ARCHIVE_NAME='2022-10-01-13.json.gz'

SEED_DIR = 'data/gharchive/seeds/apple-day0-mattyoung'
SEED_ARCHIVE = f'{SEED_DIR}/{SEED_ARCHIVE_NAME}'
print(f'\nSEED_ARCHIVE: {SEED_ARCHIVE}\n')


# intermediate files
OUT_DIR = 'data/gharchive/.processed/apple-day0-mattyoung'

# DICTIONARY CONTAINING ALL TABLES
# key: table_name, value: dataframe object
tbls = defaultdict(pd.DataFrame)

ImportError: cannot import name 'Connection' from 'sqlalchemy' (/opt/conda/lib/python3.11/site-packages/sqlalchemy/__init__.py)

In [3]:
!pip3 show sqlalchemy pandas sqlacodegen

Name: SQLAlchemy
Version: 2.0.19
Summary: Database Abstraction Library
Home-page: https://www.sqlalchemy.org
Author: Mike Bayer
Author-email: mike_mp@zzzcomputing.com
License: MIT
Location: /opt/conda/lib/python3.11/site-packages
Requires: greenlet, typing-extensions
Required-by: alembic, jupyterhub, sqlacodegen
---
Name: pandas
Version: 1.5.1
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /opt/conda/lib/python3.11/site-packages
Requires: numpy, python-dateutil, pytz
Required-by: altair, bokeh, modin, seaborn, statsmodels
---
Name: sqlacodegen
Version: 2.3.0
Summary: Automatic model code generator for SQLAlchemy
Home-page: UNKNOWN
Author: Alex Grönholm
Author-email: alex.gronholm@nextday.fi
License: MIT
Location: /opt/conda/lib/python3.11/site-packages
Requires: inflect, SQLAlchemy
Required-by: 


## Load OSRB and ecosystem data

In [4]:
osrb = load_repos('data/osrb/osrb-repos.txt')
display([dfoneline('osrb', osrb)], osrb.head(5))

osrb, memory: 25074 bytes, shape: (116, 3), cols=['name', 'org_name', 'repo_name']


[None]

,name,org_name,repo_name
0,google/angle,google,angle
1,apache/airflow,apache,airflow
2,apache/arrow,apache,arrow
3,apache/cassandra,apache,cassandra
4,apache/cloudstack,apache,cloudstack


In [5]:
osrb = safe_set_index(osrb, ['org_name', 'repo_name'])

display([dfoneline('osrb', osrb)])
osrb

osrb, memory: 27404 bytes, shape: (116, 1), cols=['name']


[None]

name
org_name                  repo_name                                                                
AcademySoftwareFoundation MaterialX                             AcademySoftwareFoundation/MaterialX
                          OpenColorIO                         AcademySoftwareFoundation/OpenColorIO
                          openexr                                 AcademySoftwareFoundation/openexr
                          openvdb                                 AcademySoftwareFoundation/openvdb
                          rez                                         AcademySoftwareFoundation/rez
Autodesk                  maya-usd                                                Autodesk/maya-usd
DefinitelyTyped           DefinitelyTyped                           DefinitelyTyped/DefinitelyTyped
GNOME                     libxml2                                                     GNOME/libxml2
                          libxslt                                                     GNOME/libxslt
GoogleCloudPlatform       spark-on-k8s-operator           GoogleCloudPlatform/spark-on-k8s-operator
PixarAnimationStudios     USD                                             PixarAnimationStudios/USD
SeleniumHQ                selenium                                              SeleniumHQ/selenium
apache                    airflow                                                    apache/airflow
                          arrow                                                        apache/arrow
                          cassandra                                                apache/cassandra
                          cloudstack                                              apache/cloudstack
                          druid                                                        apache/druid
                          flink                                                        apache/flink
                          flume                                                        apache/flume
                          hadoop                                                      apache/hadoop
                          hbase                                                        apache/hbase
                          hive                                                          apache/hive
                          iceberg                                                    apache/iceberg
                          incubator-toree                                    apache/incubator-toree
                          kafka                                                        apache/kafka
                          kyuubi                                                      apache/kyuubi
                          logging-log4j2                                      apache/logging-log4j2
                          lucene                                                      apache/lucene
                          orc                                                            apache/orc
                          ozone                                                        apache/ozone
                          parquet-mr                                              apache/parquet-mr
                          solr                                                          apache/solr
                          spark                                                        apache/spark
                          superset                                                  apache/superset
                          trafficserver                                        apache/trafficserver
                          yunikorn-core                                        apache/yunikorn-core
                          zookeeper                                                apache/zookeeper
apple                     FHIRModels                                               apple/FHIRModels
                          GCGC                                                           apple/

### osrb projects: 'org_name' = Apple

In [6]:
osrb.loc['apple']

,name
repo_name,
FHIRModels,apple/FHIRModels
GCGC,apple/GCGC
app-store-server-library-swift,apple/app-store-server-library-swift
batch-processing-gateway,apple/batch-processing-gateway
coremltools,apple/coremltools
darwin-libplatform,apple/darwin-libplatform
foundationdb,apple/foundationdb
password-manager-resources,apple/password-manager-resources
servicetalk,apple/servicetalk


### osrb projects: 'org_name' = Apache

In [7]:
osrb.loc['apache']

,name
repo_name,
airflow,apache/airflow
arrow,apache/arrow
cassandra,apache/cassandra
cloudstack,apache/cloudstack
druid,apache/druid
flink,apache/flink
flume,apache/flume
hadoop,apache/hadoop
hbase,apache/hbase


## What is the Github Archive?

https://www.gharchive.org

> Open-source developers all over the world are working on millions of projects: writing code & documentation, fixing & submitting bugs, and so forth. GH Archive is a project to record the public GitHub timeline, archive it, and make it easily accessible for further analysis.
>
>GitHub provides 15+ event types, which range from new commits and fork events, to opening new tickets, commenting, and adding members to a project. These events are aggregated into hourly archives, which you can access with any HTTP client

* Activity archives are available starting 2/12/2011.
* Activity archives for dates between 2/12/2011-12/31/2014 was recorded from the (now deprecated) Timeline API.
* **Activity archives for dates starting 1/1/2015 is recorded from the Events API.**

## Design: Github Events

### How Github Event types are modeled

> The Events API can return different types of events triggered by activity on GitHub. Each event response contains shared properties, but has a unique payload object determined by its event type. The Event object common properties describes the properties shared by all events, and each event type describes the payload properties that are unique to the specific event.

_More info: https://docs.github.com/en/webhooks-and-events/events/github-event-types_

We use python3's [support for enumerations](https://docs.python.org/3/library/enum.html) to capture the **kinds of things** that exist.

* code completion in editors reduces iterations due to typos in config, and RSI
* the functions provided by the `enum` class are powerful. The HOWTO is detailed: https://docs.python.org/3/howto/enum.html
* we leverage it for configuration


```python
class EventType(Enum):
    '''https://docs.github.com/en/webhooks-and-events/events/github-event-types '''

    def __str__(self):
            return str(self.value)

    CommitCommentEvent            = 'CommitCommentEvent'
    CreateEvent                   = 'CreateEvent'
    DeleteEvent                   = 'DeleteEvent' 
    ForkEvent                     = 'ForkEvent'
    GollumEvent                   = 'GollumEvent'
    IssueCommentEvent             = 'IssueCommentEvent'
    IssuesEvent                   = 'IssuesEvent'
    MemberEvent                   = 'MemberEvent'
    PublicEvent                   = 'PublicEvent'
    PullRequestEvent              = 'PullRequestEvent'
    PullRequestReviewEvent        = 'PullRequestReviewEvent'
    PullRequestReviewCommentEvent = 'PullRequestReviewCommentEvent'
    PullRequestReviewThreadEvent  = 'PullRequestReviewThreadEvent'
    PushEvent                     = 'PushEvent'
    ReleaseEvent                  = 'ReleaseEvent'
    SponsorshipEvent              = 'SponsorshipEvent'
    WatchEvent                    = 'WatchEvent'
```

In [8]:
for et in list(EventType):
    print(et)

CommitCommentEvent
CreateEvent
DeleteEvent
ForkEvent
GollumEvent
IssueCommentEvent
IssuesEvent
MemberEvent
PublicEvent
PullRequestEvent
PullRequestReviewEvent
PullRequestReviewCommentEvent
PullRequestReviewThreadEvent
PushEvent
ReleaseEvent
SponsorshipEvent
WatchEvent


### Entities

#### Common properties on each event


```json
[
  {
    "type": "WatchEvent",
    "public": false,
    "payload": {
    },
    "repository": {
      "id": 3,
      "name": "octocat/Hello-World",
      "url": "https://api.github.com/repos/octocat/Hello-World"
    },
    "actor": {
      "id": 1,
      "login": "octocat",
      "gravatar_id": "",
      "avatar_url": "https://github.com/images/error/octocat_happy.gif",
      "url": "https://api.github.com/users/octocat"
    },
    "org": {
      "id": 1,
      "login": "github",
      "gravatar_id": "",
      "url": "https://api.github.com/orgs/github",
      "avatar_url": "https://github.com/images/error/octocat_happy.gif"
    },
    "created_at": "2011-09-06T17:26:27Z",
    "id": "12345"
  }
]
```





In each event's common header, there are 3 interior objects, `repository`, `actor`, and `org`. Their existence is captured similarly in a python `Enum`:

```python
class EntityType(Enum):
    ''' Entities, one DataFrame per entity (table) in the resulting DB'''

    def __str__(self):
            return str(self.value)

    EventType                             = 'EventType'
    Event                                 = 'Event'
    Actor                                 = 'Actor'
    Repo                                  = 'Repo'
    Org                                   = 'Org'

    ForkEvent__Forkee                     = 'ForkEvent__Forkee'
    ForkEvent_Forkee__Owner               = 'ForkEvent_Forkee__Owner'
    ForkEvent_Forkee__License             = 'ForkEvent_Forkee__License'

    IssueCommentEvent__Issue              = 'IssueCommentEvent__Issue'
    IssueCommentEvent_Issue__Pull_Request = 'IssueCommentEvent_Issue__Pull_Request'

    IssuesEvent__Issue                     = 'IssuesEvent__Issue'
    IssuesEvent_Issue__Pull_Request        = 'IssuesEvent_Issue__Pull_Request'
    
    PullRequestEvent__PullRequest          = 'PullRequestEvent__PullRequest'
```

In [9]:
for et in list(EntityType):
    print(et)

EventType
Event
Actor
Repo
Org
ForkEvent__Forkee
ForkEvent_Forkee__Owner
ForkEvent_Forkee__License
IssueCommentEvent__Issue
IssueCommentEvent_Issue__Pull_Request
IssuesEvent__Issue
IssuesEvent_Issue__Pull_Request
PullRequestEvent__PullRequest


## Load Event Data

### Download archive (October 1st, 2022, 9am-10am ET)

In [10]:
!bash seed-apple-day0-mattyoung.sh

+ pwd
/home/jovyan
++ git rev-parse --show-toplevel
+ GIT_ROOT_DIR=/home/jovyan
+ cd /home/jovyan
+ pwd
/home/jovyan
+ SEED_ROOT=data/gharchive/seeds
+ SEED_DIR=data/gharchive/seeds/apple-day0-mattyoung
+ mkdir -p data/gharchive/seeds/apple-day0-mattyoung
+ wget --continue --directory-prefix data/gharchive/seeds/apple-day0-mattyoung https://data.gharchive.org/2022-10-01-13.json.gz
--2023-08-10 23:46:25--  https://data.gharchive.org/2022-10-01-13.json.gz
Resolving data.gharchive.org (data.gharchive.org)... 172.67.168.206, 104.21.46.175, 2606:4700:3035::ac43:a8ce, ...
Connecting to data.gharchive.org (data.gharchive.org)|172.67.168.206|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



### JSON Lines --> pandas DataFrame

* Each archive is a compressed (.gz) JSON Lines formatted file.
* Each line of the file is a JSON object representing an event that happened, and is one of 20 types.

In [11]:
dd.read_json?

Signature:
dd.read_json(
    url_path,
    orient='records',
    lines=None,
    storage_options=None,
    blocksize=None,
    sample=1048576,
    encoding='utf-8',
    errors='strict',
    compression='infer',
    meta=None,
    engine=<function read_json at 0xffff191d5c60>,
    include_path_column=False,
    path_converter=None,
    **kwargs,
)
Docstring:
Create a dataframe from a set of JSON files

This utilises ``pandas.read_json()``, and most parameters are
passed through - see its docstring.

Differences: orient is 'records' by default, with lines=True; this
is appropriate for line-delimited "JSON-lines" data, the kind of JSON output
that is most common in big-data scenarios, and which can be chunked when
reading (see ``read_json()``). All other options require blocksize=None,
i.e., one partition per input file.

Parameters
----------
url_path: str, list of str
    Location to read from. If a string, can include a glob character to
    find a set of file names.
    Supports proto

In [12]:
archive = SEED_ARCHIVE
!ls -laF '{archive}' && echo ""

#%time event = pd.read_json(archive, orient='records', lines=True)

# 256 MB
blocksize = 2**28

%time event = dd.read_json(archive, orient='records', lines=True, chunksize=blocksize)

# reduce sample size as necessary
#@event = event.sample(n=400, random_state=42)

-rw-r--r-- 1 jovyan users 76139958 Oct  1  2022 data/gharchive/seeds/apple-day0-mattyoung/2022-10-01-13.json.gz



2023-08-10 23:46:27,056 - distributed.worker - WARNING - Compute Failed
Key:       make_meta-13c9d03a-bb92-4a12-ac02-1262f5ac7eae
Function:  make_meta
args:      (<pandas.io.json._json.JsonReader object at 0xffff673d1750>)
kwargs:    {}
Exception: "ValueError('I/O operation on closed file.')"



ValueError: An error occurred while calling the read_json method registered to the pandas backend.
Original Message: I/O operation on closed file.

In [13]:
event.visualize()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

In [ ]:

event['org_repo_name'] = event.apply(lambda x: x['repo']['name'], axis = 1).astype('string')

# org/repo --> org_name, repo_name
event = split_org_repo(event, 'org_repo_name')

dfoneline('event', event)
%time event = event.astype(config['event']['dtypes_df'])
dfoneline('event', event)

print(event.dtypes)
event.head(3)

# store in global 
tbls['event'] = event

Recall that the header looks like this, note "id" is used as the a primary key as convention.  The helper library fixes up `id` columns to be named `{entity}_github_id`. 

```json
[
  {
    "type": "WatchEvent",
    "public": false,
    "payload": {
    },
    "repository": {
      "id": 3,
      "name": "octocat/Hello-World",
      "url": "https://api.github.com/repos/octocat/Hello-World"
    },
    "actor": {
      "id": 1,
      "login": "octocat",
      "gravatar_id": "",
      "avatar_url": "https://github.com/images/error/octocat_happy.gif",
      "url": "https://api.github.com/users/octocat"
    },
    "org": {
      "id": 1,
      "login": "github",
      "gravatar_id": "",
      "url": "https://api.github.com/orgs/github",
      "avatar_url": "https://github.com/images/error/octocat_happy.gif"
    },
    "created_at": "2011-09-06T17:26:27Z",
    "id": "12345"
  }
]
```

The `name` and `url` fields are specified as `category_cols` - indicating that we should handle them as a [Pandas Category](https://pandas.pydata.org/docs/user_guide/categorical.html).

>The categorical data type is useful in the following cases:
>
> * A string variable consisting of only a few different values. Converting such a string variable to a categorical variable will save some memory, see here.
>
> * The lexical order of a variable is not the same as the logical order (“one”, “two”, “three”). By converting to a categorical and specifying an order on the categories, sorting and min/max will use the logical order instead of the lexical order, see here.
>
> * As a signal to other Python libraries that this column should be treated as a categorical variable (e.g. to use suitable statistical methods or plot types).

This is similar to the "dictionary compression" used in the Arrow projects. The value is stored once per unique value, rather than being duplicated per event. This produces a dramatic reduction in memory usage, making it viable to be used by an engineer with a MacBook Pro.

More info: https://pandas.pydata.org/docs/user_guide/categorical.html#categorical-memory

Configuration for the `repo` entity:

```json
str(EntityType.Repo).lower(): {
    'description': 'ENTITY: repo',
    'dtypes_write': {
        'repo_github_id': sa_dialect_postgresql.BIGINT,
        'name':           sa_dialect_postgresql.TEXT,
        'org_name':       sa_dialect_postgresql.TEXT,
        'repo_name':      sa_dialect_postgresql.TEXT,
        'url':            sa_dialect_postgresql.TEXT
        },
    'category_cols': ['name', 'url'],
    'embedded_json_cols': [],
    'natural_keys': ['id'],
    'db_table_name': 'repo'
    },
```

## Filter out repos we don't wish to process

In [ ]:
osrb = osrb.reset_index()
dfoneline('event', event)
osrb.head(5)

In [ ]:
ecosystem_orgs = set(['adobe','alibaba','artsy','cfpb','cloudflare','docker','eleme',
                      'godaddy','google','guacsec','guardian','microsoft','mozilla',
                      'newrelic','ocsf','sigstore','square','stripe','twitter','wikimedia','zalando'])

osrb_orgs  = set(osrb['org_name'])
osrb_repos = set(osrb['name'])

# TODO: add generic filtering based on tags (leveraging postgres JSONB arrays + GIN index)

dfoneline('    event', event)

# non-ecosystem events
event_eco = event[event['org_name'].isin(osrb_orgs.union(ecosystem_orgs))]
dfoneline('  eco_org', event_eco)

event_osrb_org = event_eco[event_eco['org_name'].isin(osrb_orgs)]
dfoneline(' osrb_org', event_osrb_org)

event_osrb_repo = event_osrb_org[event_osrb_org['org_repo_name'].isin(osrb_repos)]
dfoneline('osrb_repo', event_osrb_repo)

# safe_set_index(event, ['org_name','repo_name','id'])

#     event, memory: 144755174 bytes, shape: (166151, 11)
#   eco_org, memory:  20423732 bytes, shape: (770, 11)
#  osrb_org, memory:  20343172 bytes, shape: (674, 11)
# osrb_repo, memory:  19884950 bytes, shape: (125, 11)

# SHED MEMORY
event = event_eco.copy()
display(event.info(), event.head(3))

## Common Entities

**Create new tables w/ the normalized (previously nested) JSON**

### Entity: Repo

In [ ]:
# ENTITY: repo
event, repo = nested_dimension(tbls, 'repo', event, 'repo')
display(repo.head(3))
event.head(3)

In [ ]:
#### org/repo --> org_name, repo_name

In [ ]:
repo = split_org_repo(repo, 'name')
repo.head(3)

### Entity: Actor

In [ ]:
# ENTITY: actor
event, actor = nested_dimension(tbls, 'actor', event, 'actor')
display(actor.head(3))
event.head(3)

### Entity: Org

In [ ]:
# ENTITY: org
event, org = nested_dimension(tbls, 'org', event, 'org')
display(org.head(3))
event.head(3)

In [ ]:
print('SUMMARY')
dfoneliners(tbls)

# print('TABLES')
# dfsummaries(tbls)

## Write dataframes (tables) --> files to facilitate merging.

In [ ]:
# SEED_ARCHIVE_NAME[:-8] # YYYY-MM-DD-HH.json.gz --> YYYY-MM-DD-HH
!set -x && mkdir -p '{OUT_DIR}' && ls -lF '{OUT_DIR}'

In [ ]:
import pyarrow.feather as feather

# intermediate files
OUT_DIR = 'data/gharchive/.processed/apple-day0-mattyoung'

print(f'SEED_ARCHIVE:      {SEED_ARCHIVE}')
print(f'SEED_DIR:          {SEED_DIR}')
print(f'SEED_ARCHIVE_NAME: {SEED_ARCHIVE_NAME}')
print(f'OUT_DIR:           {OUT_DIR}')

tup = os.path.splitext(SEED_ARCHIVE_NAME)
fname_root = f'{tup[0][:-8]}'                           # YYYY-MM-DD-HH.json.gz --> YYYY-MM-DD-HH

print(f'\n*** writing to files: {list(tbls.keys())}')

for k,df in tbls.items():
    fname_base     = f'{fname_root}.{k}'          # YYYY-MM-DD-HH.actor 
    dfoneline(f'{fname_base}',df)

    full_path_base = f'{OUT_DIR}/{fname_base}'
    
    # print(f'\n*** Pandas --> Pickle: {k}')
    # pickle_fname = f'{full_path_base}.pickle.gz'      # 2022-10-01.repo.pickle.gz
    # %time df.to_pickle(pickle_fname)
    # print(f'DONE writing file: {pickle_fname}')
        
    print(f'\n*** Pandas --> Arrow: {k}')
    arrow_fname = f'{full_path_base}.arrow'      # 2022-10-01.repo.arrow
    
    with open(arrow_fname, 'wb') as f:
        %time feather.write_feather(df, f, compression="lz4")
    print(f'DONE writing file: {arrow_fname}')

In [ ]:
!du -hs '{SEED_DIR}' && ls -lhS '{SEED_DIR}'/*
!du -hs '{OUT_DIR}' && ls -lhS '{OUT_DIR}'/*

pd.set_option('display.max_columns', None)
event.head()

In [ ]:
display(event.dtypes)

In [ ]:
# %run data/gharchive/gharchive.py
# config = GHARCHIVE_ENTITY_CONFIG
# event_dtypes_df = config['event']['dtypes_df']
# event_new = event.astype(event_dtypes_df)

# display(event.dtypes)
# display(event_new.dtypes)

# dfoneline('*** event    ', event)
# dfoneline('*** event_new', event_new)

# print(f'\n***  event\n-----\n')
# dfmemstat(event)
      
# print(f'\n*** event_new\n---------\n')
# dfmemstat(event_new)

# event.head(3)

In [ ]:
dfmemstat(event)

### Resources (arrow/feather/pandas)

* https://arrow.apache.org/docs/python/pandas.html
* https://arrow.apache.org/docs/python/feather.html#feather-file-format
* https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_feather.html#pandas-dataframe-to-feather
* https://arrow.apache.org/docs/python/generated/pyarrow.feather.write_feather.html#pyarrow.feather.write_feather

* https://arrow.apache.org/docs/python/data.html#dictionary-arrays

* https://observablehq.com/@uwdata/introducing-arquero
* https://observablehq.com/@uwdata/arquero-and-apache-arrow
* https://observablehq.com/@uwdata/an-illustrated-guide-to-arquero-verbs


## Push tables --> Postgres Database

In [ ]:
!pip list | grep jupyter